In [ ]:
import os
import time
from gtts import gTTS
from IPython.display import display, HTML
import pygame
import speech_recognition as sr
from difflib import SequenceMatcher

# pygame 초기화 및 폴더 생성
pygame.mixer.init()
if not os.path.exists('Learning'):
    os.makedirs('Learning')
if not os.path.exists('Speaking'):
    os.makedirs('Speaking')

# 음성 인식 초기화
r = sr.Recognizer()
mic = sr.Microphone()

def clean_char_files():
    # Learning 폴더 내의 기존 char_*.mp3 파일들을 삭제
    for file in os.listdir('Learning'):
        if file.startswith("char_") and file.endswith(".mp3"):
            os.remove(os.path.join('Learning', file))

def read_text_with_highlight(text, lang='ko'):
    """
    전체 텍스트를 HTML로 표시한 후,
    한 글자씩 천천히 강조하며 gTTS로 해당 글자를 읽어줍니다.
    각 글자가 읽어지는 시점에 해당 글자의 색상은 #ff8e03로 설정되며,
    기존 Learning 폴더 내의 char_*.mp3 파일은 삭제 후 재생성됩니다.
    """
    # 기존의 char 파일 삭제
    clean_char_files()
    
    # 전체 텍스트를 span 태그로 나누어 표시
    spans = ''.join([f"<span id='char{i}'>{char}</span>" for i, char in enumerate(text)])
    display(HTML(f"<div id='text-container' style='font-size: 40px; font-weight: bold;'>{spans}</div>"))
    
    for i, char in enumerate(text):
        if char.strip():  # 공백은 건너뛰기
            # 현재 읽는 글자만 #ff8e03 색상으로 강조 (나머지는 기본 색상 #45a049)
            display(HTML(
                f"<script>"
                f"document.querySelectorAll('span').forEach(el => el.style.color = '#45a049'); "
                f"document.getElementById('char{i}').style.color = '#ff8e03';"
                f"</script>"
            ))
            # 해당 글자를 gTTS로 음성 생성
            tts = gTTS(char, lang=lang)
            filename = os.path.join('Learning', f"char_{i}.mp3")
            tts.save(filename)
            
            # pygame으로 해당 음성 파일 재생
            pygame.mixer.music.load(filename)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                time.sleep(0.1)
            time.sleep(0.5)  # 글자 간 간격 조절

def play_full_sentence(sentence, lang='ko'):
    """
    전체 문장을 gTTS로 음성 생성 및 재생합니다.
    만약 Learning 폴더 내에 기존에 sentence.mp3 파일이 있다면 삭제 후 새로 생성합니다.
    """
    full_sentence_path = os.path.join('Learning', "sentence.mp3")
    if os.path.exists(full_sentence_path):
        os.remove(full_sentence_path)
    
    # gTTS로 전체 문장 음성 파일 생성
    tts = gTTS(sentence, lang=lang)
    tts.save(full_sentence_path)
    
    # 전체 문장을 크게 화면에 표시 (선택 사항)
    display(HTML(f"<div style='font-size: 40px; font-weight: bold; color: yellow;'>{sentence}</div>"))
    
    pygame.mixer.music.load(full_sentence_path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(0.1)

def highlight_differences(expected, recognized):
    """
    SequenceMatcher를 사용하여 예상 문장(expected)과 인식된 문장(recognized)의 차이를 찾아
    각 문장에서 틀린 부분을 강조한 HTML 문자열 두 개(하나는 예상 문장, 하나는 인식된 문장)를 반환합니다.
    
    - 예상 문장에서는 틀린 부분을 빨간색 글자(연한 분홍색 배경)로 표시합니다.
    - 인식된 문장에서는 틀린 부분을 파란색 글자(연한 하늘색 배경)로 표시합니다.
    """
    matcher = SequenceMatcher(None, expected, recognized)
    result_expected = []
    result_recognized = []
    
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            result_expected.append(expected[i1:i2])
            result_recognized.append(recognized[j1:j2])
        elif tag == 'replace':
            result_expected.append(f"<span style='color: red; background-color: #ffe6e6;'>{expected[i1:i2]}</span>")
            result_recognized.append(f"<span style='color: blue; background-color: #e6f7ff;'>{recognized[j1:j2]}</span>")
        elif tag == 'delete':
            result_expected.append(f"<span style='color: red; background-color: #ffe6e6;'>{expected[i1:i2]}</span>")
        elif tag == 'insert':
            result_recognized.append(f"<span style='color: blue; background-color: #e6f7ff;'>{recognized[j1:j2]}</span>")
    
    highlighted_expected = "".join(result_expected)
    highlighted_recognized = "".join(result_recognized)
    return highlighted_expected, highlighted_recognized

def record_and_check_sentence(sentence, lang='ko'):
    """
    사용자가 전체 문장을 따라 읽은 후 녹음한 음성을 인식하여,
    원본 문장과 정확히 일치하는지 확인합니다.
    또한, 녹음 결과를 Speaking 폴더에 저장하고, 저장된 파일을 HTML 오디오 플레이어로 표시합니다.
    일치하지 않을 경우, 예상 문장과 인식된 문장의 차이를 강조하여 보여줍니다.
    """
    print("아래 문장을 따라 읽어보세요:")
    print(sentence)
    
    with mic as source:
        # 주변 소음 보정 (약간의 시간이 소요됨)
        r.adjust_for_ambient_noise(source, duration=0.5)
        print("녹음 시작! 문장을 읽어주세요...")
        try:
            audio = r.listen(source, timeout=10)
        except Exception as e:
            print("녹음 중 문제가 발생했습니다. 다시 시도해주세요.")
            return False

    # 녹음 결과를 Speaking 폴더에 저장 (파일명: recording_타임스탬프.wav)
    timestamp = int(time.time())
    recording_filename = os.path.join('Speaking', f"recording_{timestamp}.wav")
    with open(recording_filename, "wb") as f:
        f.write(audio.get_wav_data())
    
    # 저장된 녹음 파일을 HTML 오디오 플레이어로 표시
    display(HTML(f"<audio controls src='{recording_filename}'></audio>"))
    
    try:
        recognized_text = r.recognize_google(audio, language="ko-KR")
        print("인식된 내용:", recognized_text)
        if recognized_text.strip() == sentence.strip():
            print("발음이 정확합니다!")
            return True
        else:
            print("발음이 정확하지 않습니다. 아래에서 틀린 부분을 확인해보세요:")
            # 강조된 차이점 표시 (난독증 아이들도 쉽게 확인할 수 있도록 단락 형식)
            highlighted_expected, highlighted_recognized = highlight_differences(sentence.strip(), recognized_text.strip())
            display(HTML(f"<div style='font-size: 20px; margin-top: 10px;'><strong>예상 문장:</strong> {highlighted_expected}</div>"))
            display(HTML(f"<div style='font-size: 20px; margin-top: 10px;'><strong>인식된 문장:</strong> {highlighted_recognized}</div>"))
            print("틀린 부분을 확인하고 다시 시도해주세요.")
            return False
    except sr.UnknownValueError:
        print("음성을 인식하지 못했습니다. 다시 시도해주세요.")
        return False
    except sr.RequestError as e:
        print("음성 인식 서비스에 접근할 수 없습니다:", e)
        return False

def test_sentence_pronunciation(sentence, lang='ko', attempts=3):
    """
    1. 한 글자씩 읽어주며 시각적으로 강조한 후,
    2. 전체 문장을 TTS로 재생(기존 파일은 삭제 후 새로 생성),
    3. 사용자가 문장을 따라 읽어 녹음한 결과를 원본과 비교하여 테스트합니다.
    만약 테스트에 통과하지 못하면, 강조된 형태로 예상 문장과 인식된 문장의 차이를 보여주고 다시 시도할 수 있도록 합니다.
    """
    # 1. 글자별 읽기 및 강조
    read_text_with_highlight(sentence, lang=lang)
    time.sleep(1)
    
    # 2. 전체 문장 TTS 재생
    play_full_sentence(sentence, lang=lang)
    
    # 3. 녹음 후 원본 문장과 비교 (최대 attempts회 시도)
    for i in range(attempts):
        print(f"\n시도 {i+1}회:")
        if record_and_check_sentence(sentence, lang=lang):
            print("테스트 통과!")
            return
        else:
            if i < attempts - 1:
                print("다시 시도해주세요.\n")
            else:
                print("최대 시도 횟수를 초과하였습니다.")

# 테스트 문장 및 전체 기능 실행
sentence = "난독증 아이들을 위한 게임입니다"
test_sentence_pronunciation(sentence)